# Данный ноутбук посвящен тестированию Бейзлайна

## Инициализация

In [6]:
import os
import pathlib

ROOT_PATH = pathlib.Path(__name__).resolve().parent.parent
print(ROOT_PATH)
RANDOM_SEED = 42

/Users/andrey/PycharmProjects


In [2]:
os.chdir(ROOT_PATH)
os.getcwd()


'/Users/andrey/PycharmProjects/vector-search-hse'

## Загрузка локального поиска по видео

У нас есть поисковый индекс построенный на эмбеддингах фреймов с частотой в 1 секунду.

Метчинг к видео происходит с помощью метаданных к ним и отбора наиболее релевантных отрезков по видео

Для задачи поиска эмбеддингов используется линейная формула косинусного сходства (эмбединги нормализованы). Функция - `brute_force_torch`

In [8]:
import pickle

with open('data/index.pkl', 'rb') as handle:
    index = pickle.load(handle)

with open('data/metadata.pkl', 'rb') as handle:
    meta = pickle.load(handle)

with open('data/thumbnails.pkl', 'rb') as handle:
    thumbnails_meta = pickle.load(handle)

In [18]:
from typing import Any, List, Tuple, Dict
import clip
from numpy.typing import NDArray
import numpy as np

import torch


def brute_force_query_torch(X, x, certainty_threshold):
    sims = (x @ X.t()).squeeze(0)  # shape: [N]

    mask = sims >= certainty_threshold
    filtered_indices = torch.nonzero(mask).squeeze(1)  # индексы в X
    filtered_sims = sims[filtered_indices]

    # Сортировка по убыванию
    sorted_sims, order = torch.sort(filtered_sims, descending=True)
    sorted_indices = filtered_indices[order]

    return sorted_indices, sorted_sims.float()


from pydantic import BaseModel


class VideoDescription(BaseModel):
    name: str
    path: str
    video_id: int
    frame_num: int
    fps: int
    start_pos: float
    end_pos: float
    score: float


class UsedVideo(BaseModel):
    start_pos: float
    end_pos: float
    score: float


class ModelConfig(BaseModel):
    device: str = 'cpu'
    frame_threshold: float = 0.26
    percentile: float = 0.8
    video_threshold: float = 0.5


class LocalSearchEngine:

    def __init__(
        self,
        config: ModelConfig,
        index: List[NDArray],
        meta: List[Any],
        thumbnails_meta: Dict[str, Any],
    ) -> None:
        self.model, self.preprocessor = clip.load(
            'ViT-B/32',
            device=config.device,
        )
        self.dataset = torch.tensor(np.array(index))
        self.thumbnails_meta = thumbnails_meta
        self.meta = meta
        self.all_videos = sorted(set([m[0] for m in meta]))
        self.video_to_int = {v: i for i, v in enumerate(self.all_videos)}
        self.int_to_video = {i: v for v, i in self.video_to_int.items()}
        self.meta_video_ids = torch.tensor(
            [self.video_to_int[m[0]] for m in meta],
            device='cpu',
            dtype=torch.int32,
        )
        self.meta_frame_nums = torch.tensor(
            [m[1] for m in meta],
            device='cpu',
            dtype=torch.int32,
        )
        self.frame_threshold: float = config.frame_threshold
        self.percentile: float = config.percentile
        self.video_threshold: float = config.video_threshold

    def encode_text(
        self,
        text: str
    ) -> torch.Tensor:
        with torch.no_grad():
            data = self.model.encode_text(clip.tokenize([text]))
        data /= torch.linalg.norm(data)
        return data

    def encode_image(
        self,
        file: NDArray,
    ) -> torch.Tensor:
        with torch.no_grad():
            data = self.model.encode_image(self.preprocessor(file).unsqueeze(0))

        data = torch.sign(data) * torch.pow(torch.abs(data), 0.25)
        data /= torch.linalg.norm(data)
        return data

    def query_frames(
        self,
        x,
        threshold: float,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        indices, certs = brute_force_query_torch(self.dataset, x, threshold)
        certs = certs.cpu()
        return indices, certs

    def get_videos_by_text(
        self,
        text: str,
    ) -> List:
        x = self.encode_text(text)
        return self.query_videos_by_tensor(x)

    def query_videos_by_tensor(
        self,
        x: torch.Tensor,
        limit: int = 20
    ) -> List[VideoDescription]:
        indices, certs = brute_force_query_torch(self.dataset, x, self.frame_threshold)
        certs = certs.cpu()
        video_idxs = self.meta_video_ids[indices]
        video_frames = self.meta_frame_nums[indices]

        video_descriptions = []
        used_videos = {}

        vals, order = torch.sort(video_idxs)
        targets = torch.tensor(
            [self.video_to_int[v] for v in self.all_videos],
            device=video_idxs.device,
        )
        order = order.cpu()
        left = torch.bucketize(targets, vals, right=False).cpu()
        right = torch.bucketize(targets, vals, right=True).cpu()

        for i, video in enumerate(self.all_videos):
            if left[i] == right[i]:
                continue
            args = order[left[i]:right[i]]
            cert_ = certs[order[left[i] + int((right[i] - left[i]) * (1 - self.percentile))]]
            if cert_ < self.video_threshold:
                continue
            subset = video_frames[args]
            start_ = torch.min(subset)
            end_ = torch.max(subset)
            max_frame = subset[0]
            used_videos[video] = UsedVideo(start_pos=start_.item(), end_pos=end_.item(), score=cert_.item())

            video_description = VideoDescription(
                name=video.split('/')[-1],
                path=video,
                video_id=self.video_to_int[video],
                frame_num=max_frame,
                fps=self.thumbnails_meta[video][1],
                start_pos=start_.item(),
                end_pos=end_.item(),
                score=cert_.item()
            )
            video_descriptions.append(video_description)

        video_descriptions = sorted(video_descriptions, key=lambda x: x.score, reverse=True)[:limit]

        return video_descriptions

# Загрузка тегов

В предыдущем ноутбуке мы обрабатывали теги на видео, полученные с помощью разметчика.

Теперь преобразуем их в удобную форму для сравнения с поисковой выдачей из движка


In [19]:
import pandas as pd

tags_df = pd.read_parquet('data/tags.parquet')
tags_df.head()

,video_name,tag_ru,tag_en
0,IMG_0903.MOV,космос,space
1,IMG_0903.MOV,усы,mustache
2,IMG_0903.MOV,мужчина,man
3,IMG_0903.MOV,NASA,NASA
4,IMG_0718.MOV,актёр,actor


In [20]:
video_base = set(tags_df['video_name'].values)

test_queries = tags_df.groupby('tag_en')['video_name'].agg(set).reset_index()
test_queries['ttl_videos'] = test_queries['video_name'].apply(lambda x: len(x))
test_queries['video_name'] = test_queries['video_name'].apply(list)

test_queries.sort_values(by='ttl_videos', ascending=False)


,tag_en,video_name,ttl_videos
164,man,"[IMG_1295.MOV, IMG_0758.MOV, IMG_1364.MOV, IMG...",145
288,woman,"[IMG_1087.MOV, IMG_0823.MOV, IMG_1153.MOV, IMG...",63
128,glasses,"[IMG_1295.MOV, IMG_0823.MOV, IMG_1364.MOV, IMG...",52
122,food,"[IMG_1192.MOV, IMG_1931.MOV, IMG_1262.MOV, IMG...",50
187,mustache,"[IMG_0758.MOV, IMG_1098.MOV, IMG_0909.MOV, IMG...",43
...,...,...,...
132,ground meat,[IMG_1267.MOV],1
133,guinea pig,[IMG_0958.MOV],1
134,guitar,[IMG_1125.MOV],1
135,hamburger,[IMG_0984.MOV],1


In [21]:
test_queries = test_queries[test_queries['ttl_videos'] > 10]

In [22]:
query_dict = dict(zip(test_queries['tag_en'].values, test_queries['video_name'].values))


## Векторный поиск видео

Проинициализируем нашу модель и для каждого запроса из тега сформируем поисковую выдачу


In [23]:
config = ModelConfig(
    device='cpu',
    frame_threshold=0.2,
    video_threshold=0.2,
    percentile=0.8,
)
engine = LocalSearchEngine(
    config=config,
    index=index,
    meta=meta,
    thumbnails_meta=thumbnails_meta,
)



In [24]:
results_per_query = {}
tests_per_query = {}

for text_query, relevant_items in query_dict.items():
    result = engine.get_videos_by_text(text_query)
    results_per_query[text_query] = result
    # For ranking
    predicted_items = [video.name for video in result]
    tests_per_query[text_query] = {
        'relevant': relevant_items,
        'predicted': predicted_items,
    }

In [25]:
QUERY = 'dog'
print(f'Actual:', tests_per_query[QUERY]['relevant'][:10], '...')
print(f'Predicted:', tests_per_query[QUERY]['predicted'][:10], '...')

Actual: ['IMG_0750.MOV', 'IMG_0982.MOV', 'IMG_0729.MOV', 'IMG_1558.MOV', 'IMG_1551.MOV', 'IMG_1602.MOV', 'IMG_0731.MOV', 'IMG_0955.MOV', 'IMG_0922.MOV', 'IMG_1675.MOV'] ...
Predicted: ['IMG_0935.MOV', 'IMG_1558.MOV', 'IMG_1779.MOV', 'IMG_0729.MOV', 'IMG_0731.MOV', 'IMG_1259.MOV', 'IMG_1551.MOV', 'IMG_1731.MOV', 'IMG_1023.MOV', 'IMG_1656.MOV'] ...


## Оценка качества по запросам

Для оценки качества наших моделей применим ряд метрик:

- Precision $$P = \frac{P_n(relevant)}{K}$$
- Recal $$R = \frac{P_n(relevant)}{N_{relevant}}$$
- F1_score - гармоническое среднее между Precision и Recall.
- MRR (Mean Reciprocal Rank)

$$


- Метрика ранжирования **Average Precision at K**

$AP\@k = \sum_k (R_k - R_{k-1}) P_k$


В тестах мы планируем показывать не всю выдачу, поэтому Recall и F1 будут выполнять вспомогательную роль для больших запросов и других тестов

Целевой метрикой по запросу для нас будет являться - **Average Precision at K** (подсчет кол-ва релевантных от 1 до K элементов в выдаче)


In [26]:
import numpy as np


# Реализация метрик

def precision_at_k(relevant_items: list[Any], predicted_ranked: list[Any], k=10) -> float:
    """Точность на первых k позициях"""
    if k > len(predicted_ranked):
        k = len(predicted_ranked)

    relevant_predicted = [item for item in predicted_ranked[:k] if item in relevant_items]
    return len(relevant_predicted) / k


def recall_at_k(relevant_items: list[Any], predicted_ranked: list[Any], k=10) -> float:
    """Полнота на первых k позициях"""
    if len(relevant_items) == 0:
        return 0.0

    relevant_predicted = [item for item in predicted_ranked[:k] if item in relevant_items]
    return len(relevant_predicted) / len(relevant_items)


def reciprocal_rank(relevant_items: list[Any], predicted_ranked: list[Any]) -> float:
    relevant_set = set(relevant_items)

    for i, item in enumerate(predicted_ranked):
        if item in relevant_set:
            return 1 / (i + 1)
    return 0


def f1_score_at_k(relevant_items: list[Any], predicted: list[Any], k=10):
    precision = precision_at_k(relevant_items, predicted, k)
    recall = recall_at_k(relevant_items, predicted, k)
    return 2 * (precision * recall) / (
        recall + precision + 1e-10
    )


def apk(
    actual: list[Any],
    predicted: list[Any],
    k: int = 10,
) -> float:
    """Метрика Average Precision at K"""
    if not actual:
        return 0.0

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)


def mapk(
    actual: list[list[Any]],
    predicted: list[list[Any]],
    k=10,
):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [27]:
# Утилиты для подсчета метрик
def eval_query_metrics(
    relevant: list[str],
    predicted: list[str],
    k: int = 10,
) -> float:
    results = {}

    # Бинарные метрики
    results['total_relevant'] = len(relevant)
    results['total_predicted'] = len(predicted)
    results['reciprocal_rank'] = reciprocal_rank(relevant, predicted)
    results[f'precision@{k}'] = precision_at_k(relevant, predicted, k)
    results[f'recall@{k}'] = recall_at_k(relevant, predicted, k)
    results[f'f1_score@{k}'] = f1_score_at_k(relevant, predicted, k)
    results[f'average_precision@{k}'] = apk(relevant, predicted, k)

    return results


def calc_for_queries(
    ranked_results: Dict[str, Dict[str, List[Any]]],
    k: int = 10,
) -> Dict[str, Any]:
    metrics = {}

    for query, result in ranked_results.items():
        metrics_by_query = eval_query_metrics(result['relevant'], result['predicted'], k)
        metrics[query] = metrics_by_query

    return metrics


In [28]:
K_TOP = 10

def get_metrics_for_queries(k: int = 10) -> Dict[str, Any]:
    metrics_per_query = calc_for_queries(
        tests_per_query,
        k=k,
    )

    stat_df = pd.DataFrame.from_dict(metrics_per_query, orient='index')
    stat_df.index.name = 'query'
    stat_df.reset_index(inplace=True)
    print('Results')
    return metrics_per_query, stat_df


top_10_metrics, top_10_metrics_df = get_metrics_for_queries(K_TOP)

display(top_10_metrics_df.sort_values(by=f'precision@{K_TOP}', ascending=False))

Results


,query,total_relevant,total_predicted,reciprocal_rank,precision@10,recall@10,f1_score@10,average_precision@10
22,woman,63,20,1.000000,0.9,0.142857,0.246575,0.900000
8,dog,19,20,1.000000,0.8,0.421053,0.551724,0.753214
12,glasses,52,20,1.000000,0.7,0.134615,0.225806,0.565873
15,man,145,20,1.000000,0.7,0.048276,0.090323,0.545873
2,beard,30,20,1.000000,0.6,0.200000,0.300000,0.432698
3,car,22,20,1.000000,0.6,0.272727,0.375000,0.529167
20,mustache,43,20,1.000000,0.6,0.139535,0.226415,0.440556
7,cooking,36,20,0.500000,0.6,0.166667,0.260870,0.373889
19,musical instrument,15,20,1.000000,0.6,0.400000,0.480000,0.509167
9,drawing,21,20,1.000000,0.5,0.238095,0.322581,0.377222


In [29]:
print(top_10_metrics_df.sort_values(by=f'average_precision@{K_TOP}', ascending=False).to_markdown())

|    | query              |   total_relevant |   total_predicted |   reciprocal_rank |   precision@10 |   recall@10 |   f1_score@10 |   average_precision@10 |
|---:|:-------------------|-----------------:|------------------:|------------------:|---------------:|------------:|--------------:|-----------------------:|
| 22 | woman              |               63 |                20 |          1        |            0.9 |   0.142857  |     0.246575  |              0.9       |
|  8 | dog                |               19 |                20 |          1        |            0.8 |   0.421053  |     0.551724  |              0.753214  |
| 12 | glasses            |               52 |                20 |          1        |            0.7 |   0.134615  |     0.225806  |              0.565873  |
| 15 | man                |              145 |                20 |          1        |            0.7 |   0.0482759 |     0.0903226 |              0.545873  |
|  3 | car                |               22 |

## Общие метрика качества

Для подсчета общей метрики качества предлагаем воспользоваться метриками ранжирования без сортировки релевантности отдельных элементов. Это объясняется тем, что получение такой разметки очень трудозатная
 
1. MP@K (Mean Precision @K)

2. MRR (Mean reciprocal Rank)


$$ MRR = \frac{1}{U} \sum_{u=1}^{U} \frac{1}{rank_i} $$

1. Mean Average Precision@K (MAP@K)

$$ MAP\@K = \frac{1}{K} \sum_{n=1}^{K} AP\@n $$

In [86]:
common_metrics = {}

common_metrics[f'mp@{K_TOP}'] = np.mean(top_10_metrics_df[f'precision@{K_TOP}'])
common_metrics['mrr'] = np.mean(top_10_metrics_df['reciprocal_rank'])
common_metrics[f'map@{K_TOP}'] = np.mean(top_10_metrics_df[f'average_precision@{K_TOP}'])

common_metrics_df = pd.DataFrame.from_dict(common_metrics, orient='index')
common_metrics_df.reset_index(inplace=True)
common_metrics_df.columns = ['metric', 'value']

display(common_metrics_df)


,metric,value
0,mp@10,0.417391
1,mrr,0.726501
2,map@10,0.307669


## Промежуточные результаты


Мы получили метрики бейзлайна с базовыми параметрами на основании нашего пайплайна Text-To-Image-To-Video

### Метрики по отдельным запросам

Для MRR для подавляющего большинства мы получили хороший скор

Для части запросов мы получили достаточно хороший результат в по AP@K.

Запросы dog, horse, car - дали нам выдачу больше 70% по average precision по 10 объектам.

Однако есть запросы, в которых существует совсем плохие метрики отбора - meme, People, plate

### Общие метрики

В части общих метрик, у нас получились достаточно высокие метрики по MR (0.73) и низкие для MP@k (0.43) & MAP@K (0.33). Это может объясняться специфичностью отдельных тегов и способностью эмбединнгов их воспринимать, а также человеческим фактором разметки.  
 
Тем не менее, нами был получен бейзлайн поискового движка и бейзлайн по качеству, с которым мы сможем в дальнейшем сравнивать.

### Заметки

- Проверить качество разметки и возможно исключить нерелевантные теги
- Провести валидацию и подбор threshold 